In [1]:
import pandas as pd
import numpy as np

In [2]:
df0 = pd.read_csv('./after_preprocessing_data/all_df.csv')

In [3]:
df0['환율종가'] = df0['환율종가'].str.replace(',','')
df0['환율종가'] = df0['환율종가'].astype('float')
df0 = df0.rename(columns= {'South Korea' : 'EPU'})
df0['장단기금리spread']= df0['국고채3년(평균)'] - df0['콜금리(1일물,평균)']
df0 = df0.drop(['국고채3년(평균)', '콜금리(1일물,평균)'], axis=1)

In [4]:
# '물건금액(만원)' 컬럼을 마지막으로 이동시키기
cols = df0.columns.tolist()
cols.remove('물건금액(만원)')
cols.append('물건금액(만원)')
df0 = df0[cols]

df0.head()

,자치구,계약일,건물면적(㎡),층,KOSPI종가,Fed기준금리,국제유가(Dubai)($/배럴),부동산거래량,국고채5년(평균),국고채10년(평균),...,CD91물(평균),BOK기준금리,건물연식,GDP대비가계부채비율,환율종가,금값종가,선행지수순환변동치,EPU,장단기금리spread,물건금액(만원)
0,노원구,2016-01-01,39.96,10,1961.310059,0.34,26.9,9894,1.77,2.03,...,1.67,1.5,25,83.3,1175.45,76.81,99.6,177.148845,0.14,18000
1,성동구,2016-01-01,75.37,10,1961.310059,0.34,26.9,9894,1.77,2.03,...,1.67,1.5,26,83.3,1175.45,76.81,99.6,177.148845,0.14,55200
2,금천구,2016-01-01,84.83,3,1961.310059,0.34,26.9,9894,1.77,2.03,...,1.67,1.5,14,83.3,1175.45,76.81,99.6,177.148845,0.14,36800
3,은평구,2016-01-01,101.97,9,1961.310059,0.34,26.9,9894,1.77,2.03,...,1.67,1.5,6,83.3,1175.45,76.81,99.6,177.148845,0.14,67600
4,노원구,2016-01-01,84.59,10,1961.310059,0.34,26.9,9894,1.77,2.03,...,1.67,1.5,26,83.3,1175.45,76.81,99.6,177.148845,0.14,30300


In [5]:
df0['계약일'] = pd.to_datetime(df0['계약일'])

In [6]:
train_df = df0.loc[df0['계약일'] < '2021']
test_df = df0.loc[df0['계약일'] >= '2021']

In [7]:
X = train_df.drop('물건금액(만원)', axis=1)
y = train_df['물건금액(만원)']

# X_train : 숫자형 컬럼만 
X_train = X.select_dtypes([np.number])

In [8]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.435
Method:                 Least Squares   F-statistic:                 1.940e+04
Date:                Mon, 26 Jun 2023   Prob (F-statistic):               0.00
Time:                        16:21:36   Log-Likelihood:            -5.4074e+06
No. Observations:              453170   AIC:                         1.081e+07
Df Residuals:                  453151   BIC:                         1.082e+07
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              7.217e+04   3.34e+04      2.161      0.031    6723.368    1.38e+05
건물면적(㎡)             958.5084      1.832    523.180      0.000     954.918     962.099
층                   701.4023      9.144     76.707      0.000     683.481     719.324
KOSPI종가              26.8658      0.883     30.430      0.000      25.135      28.596
Fed기준금리           -3606.9903    314.636    -11.464      0.000   -4223.667   -2990.314
국제유가(Dubai)($/배럴)   262.4917     14.849     17.677      0.000     233.388     291.596
부동산거래량               -0.4681      0.016    -28.860      0.000      -0.500      -0.436
국고채5년(평균)         -1.164e+04   2714.876     -4.286      0.000    -1.7e+04   -6314.427
국고채10년(평균)         1.084e+04   1868.242      5.803      0.000    7179.382    1.45e+04
회사채3년(평균)         -4951.1367    995.883     -4.972      0.000   -6903.037   -2999.236
CD91물(평균)         -1.332e+04   1388.729     -9.594      0.000    -1.6e+04   -1.06e+04
BOK기준금리            1.267e+04   1212.339     10.450      0.000    1.03e+04     1.5e+04
건물연식                130.0793      5.808     22.398      0.000     118.696     141.462
GDP대비가계부채비율        2109.0706     45.249     46.611      0.000    2020.385    2197.757
환율종가                 32.6107      3.012     10.827      0.000      26.708      38.514
금값종가                157.6978     17.222      9.157      0.000     123.943     191.452
선행지수순환변동치         -3789.1504    317.531    -11.933      0.000   -4411.500   -3166.800
EPU                   0.5264      1.282      0.411      0.681      -1.985       3.038
장단기금리spread       -5895.9365   1456.394     -4.048      0.000   -8750.424   -3041.449
==============================================================================
Omnibus:                   246699.689   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3609544.117
Skew:                           2.306   Prob(JB):                         0.00
Kurtosis:                      16.034   Cond. No.                     1.12e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.12e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
# train
train_df.drop('EPU', axis=1, inplace=True)

C:\Users\zino0\AppData\Local\Temp\ipykernel_19296\4188678856.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop('EPU', axis=1, inplace=True)


In [10]:
X = train_df.drop('물건금액(만원)', axis=1)
y = train_df['물건금액(만원)']
X_train = X.select_dtypes([np.number])

In [13]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.435
Method:                 Least Squares   F-statistic:                 2.054e+04
Date:                Mon, 26 Jun 2023   Prob (F-statistic):               0.00
Time:                        16:23:30   Log-Likelihood:            -5.4074e+06
No. Observations:              453170   AIC:                         1.081e+07
Df Residuals:                  453152   BIC:                         1.082e+07
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              7.295e+04   3.33e+04      2.188      0.029    7610.354    1.38e+05
건물면적(㎡)             958.5020      1.832    523.196      0.000     954.911     962.093
층                   701.4035      9.144     76.707      0.000     683.482     719.325
KOSPI종가              26.8144      0.874     30.681      0.000      25.101      28.527
Fed기준금리           -3612.6437    314.334    -11.493      0.000   -4228.730   -2996.558
국제유가(Dubai)($/배럴)   263.3506     14.701     17.914      0.000     234.537     292.164
부동산거래량               -0.4693      0.016    -29.426      0.000      -0.501      -0.438
국고채5년(평균)         -1.139e+04   2648.333     -4.301      0.000   -1.66e+04   -6199.446
국고채10년(평균)         1.075e+04   1856.435      5.793      0.000    7116.383    1.44e+04
회사채3년(평균)         -5035.1699    974.646     -5.166      0.000   -6945.447   -3124.893
CD91물(평균)         -1.337e+04   1384.143     -9.659      0.000   -1.61e+04   -1.07e+04
BOK기준금리            1.258e+04   1194.056     10.538      0.000    1.02e+04    1.49e+04
건물연식                130.0442      5.807     22.394      0.000     118.663     141.426
GDP대비가계부채비율        2110.3725     45.137     46.754      0.000    2021.904    2198.841
환율종가                 32.9795      2.875     11.471      0.000      27.344      38.614
금값종가                156.6160     17.019      9.202      0.000     123.259     189.973
선행지수순환변동치         -3798.6157    316.693    -11.995      0.000   -4419.324   -3177.907
장단기금리spread       -5975.4684   1443.467     -4.140      0.000   -8804.619   -3146.318
==============================================================================
Omnibus:                   246700.997   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3609611.912
Skew:                           2.306   Prob(JB):                         0.00
Kurtosis:                      16.035   Cond. No.                     1.12e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.12e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
def myscaler(df):
    from sklearn.preprocessing import MinMaxScaler

    df_num = df.select_dtypes([np.number])
    num_col = list(df_num.columns)

    # Scaler 객체 생성
    scaler = MinMaxScaler()
    x_num_scaled = scaler.fit_transform(df_num)

    # print('\t\t(min, max) (mean, std)')
    print('Scaled (%.2f, %.2f) (%.2f, %.2f)' %(x_num_scaled.min(), x_num_scaled.max(), x_num_scaled.mean(), x_num_scaled.std()))
    x_num_scaled = pd.DataFrame(x_num_scaled, columns=num_col)

    X_scaled = df.copy()
    for i in x_num_scaled:
        X_scaled[i] = x_num_scaled[i]

    return X_scaled

In [15]:
X_train = myscaler(X_train)
y = np.log(y)

Scaled (0.00, 1.00) (0.42, 0.27)


In [16]:
import statsmodels.api as sm

feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.502
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                 2.684e+04
Date:                Mon, 26 Jun 2023   Prob (F-statistic):               0.00
Time:                        16:25:39   Log-Likelihood:            -2.5509e+05
No. Observations:              453170   AIC:                         5.102e+05
Df Residuals:                  453152   BIC:                         5.104e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 9.5813      0.017    557.408      0.000       9.548       9.615
건물면적(㎡)               5.2096      0.009    594.683      0.000       5.192       5.227
층                     0.6624      0.008     87.167      0.000       0.647       0.677
KOSPI종가               0.5440      0.014     38.086      0.000       0.516       0.572
Fed기준금리              -0.0520      0.009     -6.052      0.000      -0.069      -0.035
국제유가(Dubai)($/배럴)     0.2460      0.010     24.569      0.000       0.226       0.266
부동산거래량               -0.1326      0.005    -26.036      0.000      -0.143      -0.123
국고채5년(평균)            -0.0322      0.044     -0.727      0.467      -0.119       0.055
국고채10년(평균)           -0.0169      0.033     -0.520      0.603      -0.081       0.047
회사채3년(평균)            -0.0973      0.014     -7.150      0.000      -0.124      -0.071
CD91물(평균)            -0.2103      0.020    -10.283      0.000      -0.250      -0.170
BOK기준금리               0.0868      0.017      5.040      0.000       0.053       0.121
건물연식                 -0.0230      0.004     -5.613      0.000      -0.031      -0.015
GDP대비가계부채비율           0.4573      0.011     42.540      0.000       0.436       0.478
환율종가                  0.1104      0.007     15.298      0.000       0.096       0.125
금값종가                  0.1792      0.011     16.202      0.000       0.158       0.201
선행지수순환변동치            -0.1108      0.011    -10.448      0.000      -0.132      -0.090
장단기금리spread          -0.0510      0.021     -2.470      0.014      -0.092      -0.011
==============================================================================
Omnibus:                    14036.533   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16026.210
Skew:                           0.409   Prob(JB):                         0.00
Kurtosis:                       3.424   Cond. No.                         188.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
X_train.drop(['국고채5년(평균)', '국고채10년(평균)'], axis=1, inplace=True)

## VIF 반복

In [18]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_engineering_XbyVIF(X_train_1):
    vif = pd.DataFrame()
    vif["VIF_Factor"] = [variance_inflation_factor(X_train_1.values,i)
                         for i in range(X_train_1.shape[1])]
    vif["Feature"] = X_train_1.columns
    return vif


In [19]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

    VIF_Factor            Feature
0     6.226369            건물면적(㎡)
1     5.073223                  층
2   124.592153            KOSPI종가
3    27.681945            Fed기준금리
4    85.027676  국제유가(Dubai)($/배럴)
5    12.231522             부동산거래량
6    53.711772          회사채3년(평균)
7   239.631901          CD91물(평균)
8   216.699551            BOK기준금리
9     5.551060               건물연식
10   44.757760        GDP대비가계부채비율
11   15.462898               환율종가
12   25.689920               금값종가
13   66.005518          선행지수순환변동치
14  136.696253        장단기금리spread


In [20]:
X_train.drop('CD91물(평균)', axis=1, inplace=True)

In [21]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

    VIF_Factor            Feature
0     6.223540            건물면적(㎡)
1     5.068482                  층
2   120.114358            KOSPI종가
3    27.395713            Fed기준금리
4    78.897509  국제유가(Dubai)($/배럴)
5    12.042551             부동산거래량
6    52.473596          회사채3년(평균)
7    60.481235            BOK기준금리
8     5.540367               건물연식
9    44.264996        GDP대비가계부채비율
10   15.084296               환율종가
11   20.763425               금값종가
12   64.905147          선행지수순환변동치
13  133.804770        장단기금리spread


In [22]:
X_train.drop('장단기금리spread', axis=1, inplace=True)

In [23]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)


    VIF_Factor            Feature
0     6.203726            건물면적(㎡)
1     5.057175                  층
2   116.756452            KOSPI종가
3    23.492250            Fed기준금리
4    66.736951  국제유가(Dubai)($/배럴)
5    11.525542             부동산거래량
6    20.034342          회사채3년(평균)
7    57.286229            BOK기준금리
8     5.532729               건물연식
9    44.152645        GDP대비가계부채비율
10   14.298677               환율종가
11   20.544349               금값종가
12   31.647913          선행지수순환변동치


In [24]:
X_train.drop('KOSPI종가', axis=1, inplace=True)

In [25]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

    VIF_Factor            Feature
0     6.187278            건물면적(㎡)
1     5.047962                  층
2    23.143158            Fed기준금리
3    65.591017  국제유가(Dubai)($/배럴)
4    10.727826             부동산거래량
5    18.304278          회사채3년(평균)
6    39.885120            BOK기준금리
7     5.522015               건물연식
8    17.744421        GDP대비가계부채비율
9    13.125268               환율종가
10   19.802918               금값종가
11   17.303923          선행지수순환변동치


In [26]:
X_train.drop('국제유가(Dubai)($/배럴)', axis=1, inplace=True)

In [27]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

    VIF_Factor      Feature
0     6.163737      건물면적(㎡)
1     5.036526            층
2    14.214287      Fed기준금리
3     7.825677       부동산거래량
4    13.626546    회사채3년(평균)
5    30.659710      BOK기준금리
6     5.507078         건물연식
7    14.331254  GDP대비가계부채비율
8    13.123086         환율종가
9    14.723285         금값종가
10   16.376359    선행지수순환변동치


In [28]:
X_train.drop('BOK기준금리', axis=1, inplace=True)

In [29]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

   VIF_Factor      Feature
0    5.842310      건물면적(㎡)
1    4.798128            층
2    4.046286      Fed기준금리
3    7.806027       부동산거래량
4   13.606024    회사채3년(평균)
5    5.134227         건물연식
6    7.599976  GDP대비가계부채비율
7   10.066798         환율종가
8   14.719091         금값종가
9   14.557880    선행지수순환변동치


In [30]:
X_train.drop('금값종가', axis=1, inplace=True)

In [31]:
vif = feature_engineering_XbyVIF(X_train)
print(vif)

   VIF_Factor      Feature
0    5.842289      건물면적(㎡)
1    4.796748            층
2    3.630869      Fed기준금리
3    6.659994       부동산거래량
4    8.394525    회사채3년(평균)
5    5.129151         건물연식
6    7.537006  GDP대비가계부채비율
7    6.579884         환율종가
8    8.662729    선행지수순환변동치


In [32]:
feature_add = sm.add_constant(X_train, has_constant='add')

# sm OLS 적합
model = sm.OLS(y , feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               물건금액(만원)   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                 5.009e+04
Date:                Mon, 26 Jun 2023   Prob (F-statistic):               0.00
Time:                        16:30:15   Log-Likelihood:            -2.5648e+05
No. Observations:              453170   AIC:                         5.130e+05
Df Residuals:                  453160   BIC:                         5.131e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           9.7427      0.005   1865.783      0.000       9.732       9.753
건물면적(㎡)         5.2183      0.009    594.181      0.000       5.201       5.236
층               0.6659      0.008     87.380      0.000       0.651       0.681
Fed기준금리         0.0559      0.003     21.488      0.000       0.051       0.061
부동산거래량         -0.0659      0.004    -17.405      0.000      -0.073      -0.058
회사채3년(평균)      -0.0488      0.004    -13.804      0.000      -0.056      -0.042
건물연식           -0.0220      0.004     -5.375      0.000      -0.030      -0.014
GDP대비가계부채비율     0.7344      0.004    193.798      0.000       0.727       0.742
환율종가           -0.0439      0.005     -8.423      0.000      -0.054      -0.034
선행지수순환변동치      -0.0338      0.004     -8.506      0.000      -0.042      -0.026
==============================================================================
Omnibus:                    14067.229   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16018.946
Skew:                           0.411   Prob(JB):                         0.00
Kurtosis:                       3.415   Cond. No.                         21.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
import numpy as np 
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size = 0.2)

from sklearn.linear_model import LinearRegression

mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test,y_pred)

print('MSE : {0:}, RMSE : {1: .3f}, MAE:{1: .3f}'.format(mse, rmse, mae))
print('Variance score : {0: .3f}'.format(r2_score(y_test, y_pred)))

MSE : 0.1808831749111309, RMSE :  0.425, MAE: 0.425
Variance score :  0.498


In [34]:
# df0['건물면적(㎡)'].describe()

count    536097.000000
mean         77.383479
std          31.241048
min          10.020000
25%          59.630000
50%          83.170000
75%          84.970000
max         424.320000
Name: 건물면적(㎡), dtype: float64